# Fantasy Football Web API to SQL example
* Example using "My Fantasy League" API to pull information about the player score history, and saving as SQLite db
* Brock Ricker
* Last Edited: 06/27/2022


In [1]:
#import modules here
import pandas as pd
import numpy as np
import requests
import json
from sqlalchemy import create_engine
import time

In [2]:
#Define function for pulling weekly scores from my fantasy API
def weekly_score_pull(years):
    weekly_scores = pd.DataFrame()
    
    for year in years:
        
        if year >= 2020:
            weeks = range(1,15)
        else:
            weeks = range(1,14)
        
        for week in weeks:
            print(f"parsing scores for week: {week}, year: {year}")
            request_url = f"https://www54.myfantasyleague.com/{year}/export?L=20896&W={week}&TYPE=weeklyResults&JSON=1"
            req = requests.get(request_url)
            req_data = req.json()
            print("connection results:",req)
            df_nested = pd.json_normalize(req_data["weeklyResults"]["matchup"],record_path=["franchise","player"])
            df_nested["week"] = week
            df_nested["year"] = year
            weekly_scores = pd.concat([weekly_scores,df_nested],ignore_index=True)
            time.sleep(1)
        print("sleeping so API doesn't lock me out")
        time.sleep(10)

    return weekly_scores

In [3]:
#pulling scores from 2017/2018 this has to be done in pieces due to the myfantasy donwload restrictions
early_scores = weekly_score_pull([2017,2018])

parsing scores for week: 1, year: 2017
connection results: <Response [200]>
parsing scores for week: 2, year: 2017
connection results: <Response [200]>
parsing scores for week: 3, year: 2017
connection results: <Response [200]>
parsing scores for week: 4, year: 2017
connection results: <Response [200]>
parsing scores for week: 5, year: 2017
connection results: <Response [200]>
parsing scores for week: 6, year: 2017
connection results: <Response [200]>
parsing scores for week: 7, year: 2017
connection results: <Response [200]>
parsing scores for week: 8, year: 2017
connection results: <Response [200]>
parsing scores for week: 9, year: 2017
connection results: <Response [200]>
parsing scores for week: 10, year: 2017
connection results: <Response [200]>
parsing scores for week: 11, year: 2017
connection results: <Response [200]>
parsing scores for week: 12, year: 2017
connection results: <Response [200]>
parsing scores for week: 13, year: 2017
connection results: <Response [200]>
sleeping

In [4]:
#pulling scores from 2019/2020
late_scores = weekly_score_pull([2019,2020])

parsing scores for week: 1, year: 2019
connection results: <Response [200]>
parsing scores for week: 2, year: 2019
connection results: <Response [200]>
parsing scores for week: 3, year: 2019
connection results: <Response [200]>
parsing scores for week: 4, year: 2019
connection results: <Response [200]>
parsing scores for week: 5, year: 2019
connection results: <Response [200]>
parsing scores for week: 6, year: 2019
connection results: <Response [200]>
parsing scores for week: 7, year: 2019
connection results: <Response [200]>
parsing scores for week: 8, year: 2019
connection results: <Response [200]>
parsing scores for week: 9, year: 2019
connection results: <Response [200]>
parsing scores for week: 10, year: 2019
connection results: <Response [200]>
parsing scores for week: 11, year: 2019
connection results: <Response [200]>
parsing scores for week: 12, year: 2019
connection results: <Response [200]>
parsing scores for week: 13, year: 2019
connection results: <Response [200]>
sleeping

In [5]:
#combine to one dataframe
all_scores = pd.concat([early_scores,late_scores],ignore_index=True)

In [6]:
#replacing blank scores with 0
all_scores["score"].fillna(0,inplace=True)

In [7]:
#converting datatype to numbers
all_scores[["shouldStart","score","id"]] = all_scores[["shouldStart","score","id"]].astype(float)
all_scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17137 entries, 0 to 17136
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   shouldStart  17137 non-null  float64
 1   status       17137 non-null  object 
 2   score        17137 non-null  float64
 3   id           17137 non-null  float64
 4   week         17137 non-null  int64  
 5   year         17137 non-null  int64  
dtypes: float64(3), int64(2), object(1)
memory usage: 803.4+ KB


In [8]:
#importing player list
request_url = f"https://www54.myfantasyleague.com/2022/export?L=20896&TYPE=players&JSON=1&DETAILS=1&SINCE=2017"
req = requests.get(request_url)
req_data = req.json()
print("connection results:",req)
players = pd.json_normalize(req_data["players"],record_path=["player"])

connection results: <Response [200]>


In [17]:
players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 931 entries, 0 to 930
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   draft_year        931 non-null    object
 1   draft_round       649 non-null    object
 2   nfl_id            193 non-null    object
 3   rotoworld_id      557 non-null    object
 4   stats_id          777 non-null    object
 5   position          931 non-null    object
 6   stats_global_id   931 non-null    object
 7   espn_id           274 non-null    object
 8   weight            921 non-null    object
 9   id                931 non-null    object
 10  birthdate         908 non-null    object
 11  draft_team        931 non-null    object
 12  name              931 non-null    object
 13  draft_pick        649 non-null    object
 14  college           927 non-null    object
 15  height            921 non-null    object
 16  rotowire_id       928 non-null    object
 17  jersey          

# SQLite

In [9]:
# create sqlite engine for a db called fantasy_league
engine = create_engine("sqlite:///fantasy_league.db", echo=True)
#create connection to the engine
conn = engine.connect()

In [13]:
#adding player list to sql db
players.to_sql("players", conn, if_exists="fail")

2022-06-27 17:28:59,393 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("players")
2022-06-27 17:28:59,394 INFO sqlalchemy.engine.Engine [raw sql] ()


ValueError: Table 'players' already exists.

In [14]:
#adding all scoring data to sql db
all_scores.to_sql("scores", conn, if_exists="fail")

2022-06-27 17:30:50,468 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("scores")
2022-06-27 17:30:50,468 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-27 17:30:50,469 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("scores")
2022-06-27 17:30:50,469 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-27 17:30:50,470 INFO sqlalchemy.engine.Engine 
CREATE TABLE scores (
	"index" BIGINT, 
	"shouldStart" FLOAT, 
	status TEXT, 
	score FLOAT, 
	id FLOAT, 
	week BIGINT, 
	year BIGINT
)


2022-06-27 17:30:50,470 INFO sqlalchemy.engine.Engine [no key 0.00026s] ()
2022-06-27 17:30:50,485 INFO sqlalchemy.engine.Engine COMMIT
2022-06-27 17:30:50,486 INFO sqlalchemy.engine.Engine CREATE INDEX ix_scores_index ON scores ("index")
2022-06-27 17:30:50,486 INFO sqlalchemy.engine.Engine [no key 0.00027s] ()
2022-06-27 17:30:50,496 INFO sqlalchemy.engine.Engine COMMIT
2022-06-27 17:30:50,606 INFO sqlalchemy.engine.Engine INSERT INTO scores ("index", "shouldStart", status, score, id, week, ye

17137

In [16]:
#example SQL query to test scores table

sql = """
SELECT shouldStart 
FROM scores
Limit 3;
"""
# Read the sql query into pandas to view the results
pd.read_sql_query(sql, engine)

2022-06-27 17:33:03,276 INFO sqlalchemy.engine.Engine 
SELECT shouldStart 
FROM scores
Limit 3;

2022-06-27 17:33:03,276 INFO sqlalchemy.engine.Engine [raw sql] ()


,shouldStart
0,1.0
1,1.0
2,0.0


In [18]:
#example SQL query to test players table

sql = """
SELECT id 
FROM players
Limit 3;
"""
# Read the sql query into pandas to view the results
pd.read_sql_query(sql, engine)

2022-06-27 17:34:40,371 INFO sqlalchemy.engine.Engine 
SELECT id 
FROM players
Limit 3;

2022-06-27 17:34:40,371 INFO sqlalchemy.engine.Engine [raw sql] ()


,id
0,5848
1,7911
2,8062
